### Accessing the Jawbone API using OAuth2

Next, we will create the code that can interact with the Jawbone API. The Jawbone API documentation (https://jawbone.com/up/developer/) provides a [good explanation of the authentication process](https://jawbone.com/up/developer/authentication). 

Below, we implement the OAuth2 flow for the API. The code below will create the webserver that will handle the "callback" calls from the API. (Steps 3 and 4 from the figure above.) Then, we will show how to implement the rest of the flow.

In [1]:
# This part creates a temporary web server to "receive" the requests by Jawbone

# In this cell, we just configure the server. We will start it, 
# using the command webserver.run(host='0.0.0.0') in the next cell

# Flask is a webserver
from flask import Flask, request

# We will use that part to issue a request to Jawbone
import requests
import json



# Get these after registering an app with Jawbone
# These are to communicate to Jawbone that the requests come
# from a legitimate, registered app
CLIENT_ID = 'F3rPnz_fTUY'
CLIENT_SECRET = 'efaedc99a6bc5fb40f4be3b8fb97c6b94a7eec58'

# This is the location where we will store the authentication data from Jawbone
OAUTH_FILE = '/home/ubuntu/data/jawbone_secret.json'

# Initialize the embedded web server
webserver = Flask("JawboneOAuth")

# The function below will implement the sole functionality of our server.
# The Jawbone API will first authenticate the user and then direct the browser
# to http://<your ip>:5000/receiveCode in order to send the code to the app.
# 
# This is the place where the webserver will receive the call from Jawbone
# with the authentication code (Step 3 in https://jawbone.com/up/developer/authentication)
# Notice that we will use the http://<your ip>:5000/receiveCode address below as the redirect URI 
@webserver.route("/receiveCode")
def oauth_helper():
    code = request.args.get('code')
    
    # Now that we got the code, we move to Step 4 of the instructions
    # and request the access token from Jawbone. Notice that we 
    # use the client_secret to prove that the app is the real one
    # that was registered with the Jawbone API
    url = "https://jawbone.com/auth/oauth2/token"
    params = {"grant_type": "authorization_code", 
              "client_id": CLIENT_ID, 
              "client_secret": CLIENT_SECRET, 
              "code": code }
    resp = requests.get(url, params=params)
    data = json.loads(resp.text)
    
    # We store the code in a file as the webserver does not interact with the 
    # rest of the Python code, and we also want to reuse the code in the future
    # (Typically, we would store the access_token in a database.)
    f = open(OAUTH_FILE, 'w') # Store the code as a file
    f.write(resp.text)
    f.close()
    
    # It is safe (and convenient) to shut down the web server after this request
    stop_server()
    
    # What we return here has no real impact on the functionality of the code
    return '<html><body>Code: <b>'+code+'</b><p>Response:<b>'+resp.text+'</b></body></html>'


def start_server():
    webserver.run(host='0.0.0.0', port=5000)
    return
    
def stop_server():
    shutdown_after_request = request.environ.get('werkzeug.server.shutdown')
    shutdown_after_request()
    return
    


So our server is ready to run now, and it's ready to accept the requests from Jawbone. In principle, we can write the code in the cell above in a separate Python script, and let it run in the background.

Let's start the OAuth2 flow now, to first interact with the user and get the user to login and grant permissions. (Steps 1 and 2 of the figure above.)

In [ ]:
# To get the server to run in the background
import threading

# This part is required to come up with the pop-up with which Jawbone will ask for permissions
from IPython.display import display
from IPython.display import Javascript as JS


# Send an OAuth request to Jawbone, handle the redirect, and display the access
# token that's included in the redirect for the user to copy and paste
REDIRECT_URI = 'http://ipython.ipeirotis.com:5000/receiveCode'
JAWBONE_URL = 'https://jawbone.com/auth/oauth2/auth'
PERMISSIONS = 'basic_read extended_read sleep_read move_read'

url = (JAWBONE_URL + 
       '?response_type=code&client_id='+ CLIENT_ID + 
       '&scope=' + PERMISSIONS +
       '&redirect_uri=' + REDIRECT_URI )

# The code below is necessary to pop up the login window from Jawbone
# The login still start in a separate thread, in order to allow the execution
# to continue, and run the "start_server()" call below.
threading.Timer(1, lambda: display(JS("window.open('%s')" % url))).start()

# And now start the webserver so that we can receive the answer from Jawbone API
start_server()

 * Serving Flask app "JawboneOAuth" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


<IPython.core.display.Javascript object>

In [ ]:
# Read the access token from the file
f = open(OAUTH_FILE, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)

auth_info

At this point we are pretty much done. We only need to send standard API calls, as usual, with the only addition being that we include an extra _header_ in the request, with the access_token. The code below is just showing various use cases of the API.

In [ ]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/moves"

# See https://jawbone.com/up/developer/endpoints/moves for details
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
moves_data = json.loads(resp.text)
moves_data

In [ ]:
import pandas as pd

df = pd.DataFrame(moves_data["data"]["items"])
df

In [ ]:
df_details = pd.DataFrame(list(df["details"]))
df_details

In [ ]:
# Testing another of the API endpoints
endpoint = "https://jawbone.com/nudge/api/v.1.1/users/@me/sleeps"

# See https://jawbone.com/up/developer/endpoints/sleeps
params = {}
headers = {'Authorization': 'Bearer ' + auth_info['access_token']}
resp = requests.get(endpoint, params=params, headers=headers)
sleep_data = json.loads(resp.text)
sleep_data

In [ ]:
df = pd.DataFrame(sleep_data["data"]["items"])
df

In [ ]:
df_details = pd.DataFrame(list(df["details"]))
df_details

In [ ]:
df_details["asleep_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["asleep_time"])
df_details["awake_time"] = map(lambda x : datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), df_details["awake_time"])
df_details